# 2. LIMPIEZA DE DATOS

1. **Renombrar y eliminar variables**
    - Usar formato `snake_case`
    - Nombres consistentes y en el mismo idioma
    - Eliminar caracteres especiales

2. **Tipos de datos correctos**
    - Convertir números almacenados como texto.
    - Parsear fechas correctamente.
    - Detectar variables categóricas mal codificadas.
    - Asegurar que los identificadores sean strings.

3. **Rango de valores válidos**
    - Eliminar valores negativos imposibles.
    - Validar porcentajes (0–100 o 0–1).
    - Revisar fechas lógicas (no futuras o irreales).
    - Verificar coordenadas y límites máximos definidos.

4. **Duplicados**
    - Detectar exactos y parciales.
    - Definir criterio de retención (más reciente o completo).
    - Registrar cantidad y método aplicado.

5. **Estandarización de categorías**
    - Convertir texto a minúsculas.
    - Corregir acentos y errores tipográficos.
    - Homogeneizar booleanos (“sí/no”, “1/0”).

6. **Agrupación de categorías**
    - Unificar sinónimos (ej. “auto”, “carro” → “automóvil”).
    - Agrupar jerárquicamente (“tablet”, “celular” → “electrónica”).
    - Documentar los mapeos originales y unificados.

7. **Alta cardinalidad**
    - Cuantificar categorías únicas.
    - Agrupar categorías raras (<5%) en “Otros”.
    - Considerar jerarquías (“ciudad → región → país”).
    - Documentar decisiones y su impacto.

8. **Validación de la variable objetivo**
    - Confirmar que no haya valores faltantes.
    - Revisar rango lógico.
    - Verificar distribución y representatividad.
    - Detectar valores extremos sospechosos.

**Acciones:** eliminar, truncar, corregir o marcar.


8. **Eliminacion de Filas con muchos NA**: Verificar la cantidad de NA tolerables por filas reduciendo la cantidad de filas perdidas pensando en una imputación futura

## 0. Carga de Datos

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 500)
pd.set_option('display.float_format', '{:.2f}'.format)

In [5]:
# Leer de nuevo el parquet
icfes = pd.read_parquet("C:/Users/CACTU/Downloads/Proyectos/prediccion-icfes-colombia/data/raw/icfes.parquet")
print(icfes.head(4))

  ESTU_TIPODOCUMENTO ESTU_NACIONALIDAD ESTU_GENERO ESTU_FECHANACIMIENTO  \
0                 TI          COLOMBIA           M           18/02/2002   
1                 TI          COLOMBIA           M           28/06/2003   
2                 TI          COLOMBIA           M            2/06/2001   
3                 TI          COLOMBIA           M           26/04/2002   

   PERIODO  ESTU_CONSECUTIVO ESTU_ESTUDIANTE ESTU_TIENEETNIA ESTU_PAIS_RESIDE  \
0    20194  SB11201940031558      ESTUDIANTE              No         COLOMBIA   
1    20194  SB11201940303339      ESTUDIANTE              No         COLOMBIA   
2    20194  SB11201940255017      ESTUDIANTE              No         COLOMBIA   
3    20194  SB11201940154799      ESTUDIANTE              Si         COLOMBIA   

  ESTU_ETNIA ESTU_DEPTO_RESIDE ESTU_COD_RESIDE_DEPTO    ESTU_MCPIO_RESIDE  \
0          -         MAGDALENA                    47            SANTA ANA   
1          -            BOGOTÁ                    11          BO

## 1. Renombrar Variables

In [6]:
### Renombramos las variables
icfes.columns = ["nacionalidad", "genero", "fecha_nac", "pertenece_etnia", "etnia_estudiante", "dpto_est_reside", 
                 "mpio_est_reside", "estrato_casa", "num_personas_casa", "num_cuartos_casa", "nivel_edu_padre", 
                 "nivel_edu_madre", "actividad_madre", "actividad_padre", "internet", "tv", "computador", "lavadora", 
                 "microndas", "carro", "moto", "consola", "num_libros", "freq_leche_derivados", "freq_carne_pescado_similares",
                 "freq_cereales_frutos_legumbres", "situacion_economica_casa", "tiempo_lectura", "tiempo_internet", 
                 "horas_trabajo_semanal", "tipo_remuneracion", "colegio_genero", "colegio_naturaleza", "colegio_calendario", 
                 "colegio_caracter", "colegio_area", "colegio_jornada", "colegio_mpio", "colegio_dpto", 
                 "est_mpio_presento_examen", "est_dpto_presento_examen", "puntaje_lectura", "percentil_lectura", 
                 "nivel_lectura", "puntaje_matematicas", "percentil_matematicas", "nivel_matematicas", "puntaje_naturales", 
                 "percentil_naturales", "nivel_naturales", "puntaje_sociales", "percentil_sociales", "nivel_sociales", 
                 "puntaje_ingles", "percentil_ingles", "nivel_ingles", "puntaje_global", "percentil_global", "inse_estudiante", 
                 "nse_estudiante", "nse_colegio"]

icfes.head(4)

ValueError: Length mismatch: Expected axis has 82 elements, new values have 61 elements

In [ ]:
# Convertir nombres a snake_case y eliminar caracteres especiales
icfes.columns = [re.sub(r'\W+', '_', col.strip().lower()) for col in icfes.columns]

# Eliminar guiones bajos múltiples y al inicio/final
icfes.columns = [re.sub(r'_+', '_', col).strip('_') for col in icfes.columns]
print("Columnas renombradas:")
print(icfes.columns.tolist())

In [ ]:
for col in icfes.select_dtypes(include=["int64", "float64"]).columns:
    n_unique = icfes[col].nunique()
    if n_unique < 20:  # arbitrario, pocos valores únicos
        print(f"⚠️ {col} es numérica pero parece categórica ({n_unique} valores únicos)")

## Outliers Variables Objetivo

In [ ]:
import numpy as np

# Extraer variable
y = icfes['PUNT_GLOBAL'].dropna()

# Normalización (Z-score)
mu, sigma = y.mean(), y.std()
z_scores = (y - mu) / sigma

# Índices de los outliers
outlier_idx = y.index[abs(z_scores) > 3]

# Guardar como lista de tuplas (índice, valor, z-score)
outliers = [(i, y.loc[i], z_scores.loc[i]) for i in outlier_idx]

print("Número de outliers:", len(outliers))
print("Ejemplo:", outliers[:5])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

plt.figure(figsize=(16,10))

# Histograma con KDE
sns.histplot(y, kde=True, stat="density", bins=30, color="lightgreen", edgecolor="black")

# Curva normal teórica
xmin, xmax = plt.xlim()
xx = np.linspace(xmin, xmax, 200)
plt.plot(xx, norm.pdf(xx, mu, sigma), 'r', lw=2, label="Normal ajustada")

# Líneas verticales de 1σ, 2σ, 3σ
for k, col in zip([1,2,3], ["black","orange","red"]):
    plt.axvline(mu + k*sigma, color=col, linestyle="--")
    plt.axvline(mu - k*sigma, color=col, linestyle="--", label=f"{k}σ")

# Graficar puntos outliers
plt.scatter(y.loc[outlier_idx], [0]*len(outlier_idx), 
            color="blue", marker="x", s=60, label="Outliers")

plt.title("Distribución de PUNT_GLOBAL con detección de outliers")
plt.xlabel("Puntaje Global")
plt.ylabel("Densidad")
plt.legend()
plt.show()

## 5. Estandarización de Categorías

In [ ]:
nom_propio_cat = ["nacionalidad", "dpto_est_reside", "mpio_est_reside", "num_libros", "colegio_genero", "colegio_naturaleza",
                  "colegio_caracter", "colegio_area", "colegio_jornada", "colegio_mpio", "colegio_dpto", "est_mpio_presento_examen", 
                  "est_dpto_presento_examen"]

icfes[nom_propio_cat] = icfes[nom_propio_cat].apply(lambda x: x.str.title() if x.dtype == "object" else x)
icfes.head(4)

## 6. Agrupamiento de Categorías

### 6.1 Reemplazar el Símbolo "-" por NAN

In [ ]:
### Cambiando el simbolo "-" por NaN para poder imputar después
# Lista para almacenar las columnas que contienen "-"
columnas_con_guion = [col for col in icfes.columns if "-" in icfes[col].astype(str).unique()]
print(columnas_con_guion)  # 🔍 Muestra las columnas que contienen "-"

for col in columnas_con_guion:
    icfes[col] = icfes[col].replace({"-": np.nan})

In [ ]:
### Insertando NA's
icfes["genero"] = icfes["genero"].replace({'M': 'Masculino', 'F': 'Femenino'})

# Verificar los valores después del reemplazo
print(icfes["genero"].value_counts(dropna=False))  # Muestra también los NaN

### 6.2 Variables del Estudiante

In [ ]:
icfes.sample(4)

In [ ]:
icfes['nse_estudiante'].value_counts(dropna=False)

In [ ]:
icfes['nse_estudiante'] = icfes['nse_estudiante'].astype('float64')

In [ ]:
icfes["pertenece_etnia"].value_counts().sort_index()

In [ ]:
icfes["etnia_estudiante"].value_counts(dropna = False).sort_index()

In [ ]:
icfes["etnia_estudiante"] = icfes["etnia_estudiante"].replace({
    np.nan: "No",
    'Comunidad afrodescendiente': 'Afro',
    'Comunidades Rom (Gitanas)': 'Gitano',
    'Ninguno': 'No',
    'Otro grupo étnico minoritario': 'Otra'
    })

icfes["etnia_estudiante"].value_counts(dropna = False).sort_index()

In [ ]:
icfes['ESTU_FECHANACIMIENTO'] = pd.to_datetime(
    icfes['ESTU_FECHANACIMIENTO'],
    format='mixed',
    dayfirst=True,
    errors='coerce'
)

icfes['ESTU_FECHANACIMIENTO'].head()
icfes['ESTU_FECHANACIMIENTO'].isna().mean() * 100  # porcentaje de fechas no convertidas

In [ ]:
# Convertir la columna a string primero para asegurar que podemos manipular el texto
icfes['ESTU_INSE_INDIVIDUAL'] = icfes['ESTU_INSE_INDIVIDUAL'].astype(str)

# Eliminar puntos y caracteres no numéricos
icfes['ESTU_INSE_INDIVIDUAL'] = icfes['ESTU_INSE_INDIVIDUAL'].str.replace('.', '')
icfes['ESTU_INSE_INDIVIDUAL'] = icfes['ESTU_INSE_INDIVIDUAL'].str.extract('(\d+)', expand=False)

# Tomar los primeros 4 dígitos y formatear como XX.XX
icfes['ESTU_INSE_INDIVIDUAL'] = icfes['ESTU_INSE_INDIVIDUAL'].str[:4]
icfes['ESTU_INSE_INDIVIDUAL'] = icfes['ESTU_INSE_INDIVIDUAL'].str[:2] + '.' + icfes['ESTU_INSE_INDIVIDUAL'].str[2:]

# Convertir a float
icfes['ESTU_INSE_INDIVIDUAL'] = pd.to_numeric(icfes['ESTU_INSE_INDIVIDUAL'], errors='coerce')

In [ ]:
def codificacion_dummy(df):
    # Renombrar columnas
    df.rename(columns={'nacionalidad': 'colombiano', 'etnia_estudiante': 'etnia'}, inplace=True)
    
    # Codificación binaria
    df['colombiano'] = (df['colombiano'] == 'Colombia').astype(int)
    df['etnia'] = (df['etnia'] != 'No').astype(int)
    
    return df

### 6.3 Variables del Hogar

In [ ]:
icfes["estrato_casa"].value_counts(dropna = False).sort_index()

In [ ]:
icfes["estrato_casa"] = icfes["estrato_casa"].replace({
    'Estrato 1': 1,
    'Estrato 2': 2,
    'Estrato 3': 3,
    'Estrato 4': 4,
    'Estrato 5': 5,
    'Estrato 6': 6,
    'Sin Estrato': np.nan,
    })

In [ ]:
icfes["num_personas_casa"].value_counts()

In [ ]:
icfes["num_personas_casa"] = icfes["num_personas_casa"].replace({
    '1 a 2': "Hogar tradicional", 
    '3 a 4': "Hogar tradicional", 
    '5 a 6': "Hogar grande", 
    '7 a 8': "Más de 1 hogar", 
    '9 o más': "Más de 1 hogar"
})

icfes["num_personas_casa"].value_counts()

In [ ]:
icfes['num_cuartos_casa'].value_counts(dropna = False)

In [ ]:
icfes['num_cuartos_casa'] = icfes['num_cuartos_casa'].replace({
    'Uno': 1,
    'Dos': 2,
    'Tres': 3,
    'Cuatro': 4,
    'Cinco': 5,
    'Seis o mas': 6
})

icfes['num_cuartos_casa'].value_counts(dropna = False).sort_values(ascending=True)

In [ ]:
icfes["internet"].value_counts(dropna = False)

In [ ]:
icfes["computador"].value_counts(dropna = False)

In [ ]:
icfes["lavadora"].value_counts(dropna = False)

In [ ]:
icfes["microndas"].value_counts(dropna = False)

In [ ]:
icfes["carro"].value_counts(dropna = False)

In [ ]:
icfes["moto"].value_counts(dropna = False)

In [ ]:
icfes["consola"].value_counts(dropna = False)

In [ ]:
columnas = ['pertenece_etnia', 'internet', 'tv', 'computador', 'lavadora', 'microndas', 'carro', 'moto', 'consola']

for col in columnas:
    icfes[col] = np.where(icfes[col] == 'Si', 1, 0)

In [ ]:
icfes.sample(6)

### 6.4 Variables de Educación y Trabajo en el Hogar

In [ ]:
icfes["nivel_edu_madre"].value_counts(dropna = False).sort_index(ascending = True)

In [ ]:
icfes["nivel_edu_madre"] = icfes["nivel_edu_madre"].replace({
  'Educación profesional completa': 'Profesional',
  'Educación profesional incompleta': 'Profesional Inc',
  'Ninguno': 'Ninguna',
  'No Aplica': 'No aplica',
  'No sabe': np.nan,
  'Primaria completa': 'Primaria',
  'Primaria incompleta': 'Primaria Inc',
  'Secundaria (Bachillerato) completa': 'Bachiller',
  'Secundaria (Bachillerato) incompleta': 'Bachiller Inc',
  'Técnica o tecnológica completa': 'Tecnico/Tecnologo',
  'Técnica o tecnológica incompleta': 'Tecnico/Tecnologo Inc'
})

icfes["nivel_edu_madre"].value_counts(dropna = False).sort_index(ascending = True)

In [ ]:
icfes["nivel_edu_padre"].value_counts(dropna = False).sort_index(ascending = True)

In [ ]:
icfes["nivel_edu_padre"] = icfes["nivel_edu_padre"].replace({
  'Educación profesional completa': 'Profesional',
  'Educación profesional incompleta': 'Profesional Inc',
  'Ninguno': 'Ninguna',
  'No Aplica': 'No aplica',
  'No sabe': np.nan,
  'Primaria completa': 'Primaria',
  'Primaria incompleta': 'Primaria Inc',
  'Secundaria (Bachillerato) completa': 'Bachiller',
  'Secundaria (Bachillerato) incompleta': 'Bachiller Inc',
  'Técnica o tecnológica completa': 'Tecnico/Tecnologo',
  'Técnica o tecnológica incompleta': 'Tecnico/Tecnologo Inc'
})

icfes["nivel_edu_padre"].value_counts(dropna = False).sort_index(ascending = True)

In [ ]:
icfes["actividad_madre"].value_counts(dropna = False).sort_index()

In [ ]:
# Definir el mapeo de categorías
mapeo = {
    # Profesionales y Directivos
    'Es dueño de un negocio grande, tiene un cargo de nivel directivo o gerencial': 'Directivos',
    'Trabaja como profesional (por ejemplo médico, abogado, ingeniero)': 'Profesionales',

    # Trabajadores Independientes y Pequeños Empresarios
    'Es dueño de un negocio pequeño (tiene pocos empleados o no tiene, por ejemplo tienda, papelería, etc': 'Microempresario',
    'Trabaja por cuenta propia (por ejemplo plomero, electricista)': 'Trabajador Independiente',

    # Trabajadores Operativos y de Servicios
    'Es operario de máquinas o conduce vehículos (taxita, chofer)': 'Trabajadores Operativos',
    'Es vendedor o trabaja en atención al público': 'Trabajadores Operativos',
    'Tiene un trabajo de tipo auxiliar administrativo (por ejemplo, secretario o asistente)': 'Trabajadores Operativos',
    'Trabaja como personal de limpieza, mantenimiento, seguridad o construcción': 'Trabajadores Operativos',
    
    # Trabajadores del Sector Primario
    'Es agricultor, pesquero o jornalero': 'Sector Primario',
    
    # Sin Actividad Laboral Remunerada
    'Trabaja en el hogar, no trabaja o estudia': 'Sin Actividad Remunerada',
    'Pensionado': 'Pensionado',
    
    # Datos No Disponibles
    'No aplica': 'No Aplica',
    'No sabe': 'Sin Información'
}

# Aplicar la reclasificación directamente a cada columna
icfes['actividad_madre'] = icfes['actividad_madre'].map(mapeo)
icfes['actividad_padre'] = icfes['actividad_padre'].map(mapeo)

In [ ]:
icfes['actividad_padre'].value_counts(dropna = False)

In [ ]:
icfes.head(4)

In [ ]:
icfes["horas_trabajo_semanal"].value_counts()

In [ ]:
icfes["horas_trabajo_semanal"] = icfes["horas_trabajo_semanal"].replace({'0': 'No Trabaja',
        'Menos de 10 horas': 'Trabajo Ocasional',
        'Entre 11 y 20 horas': 'Tiempo Parcial Reducido',
        'Entre 21 y 30 horas': 'Medio Tiempo',
        'Más de 30 horas': 'Tiempo Completo'
})

icfes["horas_trabajo_semanal"].value_counts()

In [ ]:
icfes["tipo_remuneracion"].value_counts()

In [ ]:
icfes["tipo_remuneracion"] = icfes["tipo_remuneracion"].replace({'No': 'Sin Remuneración',
        'Si, en efectivo': 'Remuneración Monetaria',
        'Si, en especie': 'Remuneración No Monetaria',
        'Si, en efectivo y especie': 'Remuneración Mixta'})

icfes["tipo_remuneracion"].value_counts()

In [ ]:
# Si "horas_trabajo_semanal" es "No Trabaja", cambiar "tipo_remuneracion" a "No Aplica"
icfes.loc[icfes["horas_trabajo_semanal"] == "No Trabaja", "tipo_remuneracion"] = "No Aplica"

### 6.5 Variables de Entretenimiento

In [ ]:
icfes.head(4)

In [ ]:
icfes["num_libros"].value_counts(dropna = False)

In [ ]:
icfes["num_libros"] = icfes["num_libros"].replace({'0 A 10 Libros': '0-10', '11 A 25 Libros': '11-25',
                                                   '26 A 100 Libros': '26-100', 'Más De 100 Libros': '>100'})

icfes["num_libros"].value_counts(dropna = False)

In [ ]:
icfes["tiempo_lectura"].value_counts(dropna = False)

In [ ]:
icfes["tiempo_lectura"] = icfes["tiempo_lectura"].replace({'No leo por entretenimiento': 'No lee', '30 minutos o menos': '0-30 min',
                                                           'Entre 30 y 60 minutos': '30-60 min', 'Entre 1 y 2 horas': '1-2h', 'Más de 2 horas': '>2h'})

icfes["tiempo_lectura"].value_counts(dropna = False)

In [ ]:
icfes["tiempo_internet"].value_counts(dropna = False)

In [ ]:
icfes["tiempo_internet"] = icfes["tiempo_internet"].replace({'No Navega Internet': 'Ninguno', '30 minutos o menos': 'Poco',
                                                             'Entre 30 y 60 minutos': 'Moderado', 'Entre 1 y 3 horas': 'Promedio', 'Más de 3 horas': 'Mucho'})

icfes["tiempo_internet"].value_counts(dropna = False)

### 6.6 Variables de Alimentación

In [ ]:
icfes.head(5)

In [ ]:
icfes["freq_leche_derivados"].value_counts()

In [ ]:
icfes["freq_carne_pescado_similares"].value_counts()

In [ ]:
icfes["freq_cereales_frutos_legumbres"].value_counts()

In [ ]:
mapping = {
    'Nunca o rara vez comemos eso': 'Muy Insuficiente',
    '1 o 2 veces por semana': 'Insuficiente',
    '3 a 5 veces por semana': 'Aceptable',
    'Todos o casi todos los días': 'Óptimo'
}

cols = ["freq_leche_derivados", 
        "freq_carne_pescado_similares", 
        "freq_cereales_frutos_legumbres"]

icfes[cols] = icfes[cols].replace(mapping)
icfes.sample(8)

In [ ]:
icfes.head(5)

### 6.7 Variables de Colegios

In [ ]:
icfes.head(4)

In [ ]:
icfes["colegio_genero"].value_counts(dropna = False)

In [ ]:
icfes["colegio_naturaleza"].value_counts(dropna = False)

In [ ]:
icfes["colegio_calendario"] = icfes["colegio_calendario"].str.capitalize()

In [ ]:
icfes["colegio_calendario"].value_counts(dropna = False)

In [ ]:
icfes["colegio_caracter"].value_counts(dropna = False)

In [ ]:
icfes["colegio_area"].value_counts(dropna = False)

In [ ]:
icfes["colegio_jornada"].value_counts(dropna = False)

### 6.8 Variables de Fecha y Edad

In [ ]:
# 1. Corregir fechas que inician con "000"
icfes["fecha_nac"] = icfes["fecha_nac"].astype(str).str.replace(r"^000", "2", regex=True)

# 2. Convertir la columna a tipo fecha manejando distintos formatos
icfes["fecha_nac"] = pd.to_datetime(icfes["fecha_nac"], errors="coerce", dayfirst=True)

# 3. Extraer solo el año
icfes["año_nacimiento"] = icfes["fecha_nac"].dt.year

# 4. Contar cuántas veces aparece cada año
conteo_fechas = icfes["año_nacimiento"].value_counts().sort_index()

# 5. Crear la variable Edad
icfes["edad"] = 2019 - icfes["año_nacimiento"]

# 6. Quitamos los decimales
icfes["edad"] = icfes["edad"].round(0).astype("Int64")

# Mostrar resultados
print(conteo_fechas)

In [ ]:
icfes["edad"].value_counts(dropna = False)

In [ ]:
plt.figure(figsize = (12, 10))
sns.countplot(x = 'edad', data = icfes)
sns.set_theme(style = 'white', context = 'talk')

plt.title('Distribución de Edades')
plt.xlabel('Edades', fontsize = 12)
plt.ylabel('Cantidad', fontsize = 12)
plt.xticks(rotation = 45, ha = 'right')
plt.tight_layout()
plt.show()

In [ ]:
## Nos quedamos solo con las edades entre 14 y 79 años
icfes = icfes[(icfes["edad"] >= 14) & (icfes["edad"] <= 79)]

In [ ]:
# Definir los límites de los bins
bins = list(range(15, 81, 5))  # De 15 a 80 en pasos de 5

# Crear etiquetas para los bins
labels = [f"{bins[i]}-{bins[i+1]-1}" for i in range(len(bins)-1)]

# Crear la nueva variable 'grupo_edad'
icfes["grupo_edad"] = pd.cut(icfes["edad"], bins=bins, labels=labels, right=False)

# Verificar distribución
print(icfes["grupo_edad"].value_counts(dropna = False))

In [ ]:
plt.figure(figsize = (12, 10))
sns.countplot(x = 'grupo_edad', data = icfes)
sns.set_theme(style = 'white', context = 'talk')

plt.title('Distribución de Edades')
plt.xlabel('Edades', fontsize = 12)
plt.ylabel('Cantidad', fontsize = 12)
plt.xticks(rotation = 45, ha = 'right')
plt.tight_layout()
plt.show()

In [ ]:
icfes["grupo_edad"].value_counts(dropna = False)

In [ ]:
# Crear la variable
icfes['presento_fuera_edad'] = (icfes['edad'] > 19).astype(int)

# Verificar distribución
print("Distribución de la variable:")
print(icfes['presento_fuera_edad'].value_counts())
print('------------------------------------------------------')
print(f"\nPorcentaje fuera de edad: {icfes['presento_fuera_edad'].mean():.2%}")
print('------------------------------------------------------')

# Verificar algunos casos
print("\nVerificación manual:")
print(icfes[['edad', 'presento_fuera_edad']].head(10))

# Estadísticas por grupo
print("\nEdad promedio por grupo:")
print(icfes.groupby('presento_fuera_edad')['edad'].agg(['count', 'mean', 'min', 'max']))

### 6.9 Variables Geográficas

In [ ]:
icfes['colegio_dpto'].value_counts(dropna = False)

In [ ]:
# Crear un diccionario con la asignación de departamentos a regiones
departamento_a_region = {
    # Región Andina
    'Antioquia': 'Andina', 'Boyaca': 'Andina', 'Caldas': 'Andina', 'Cundinamarca': 'Andina',
    'Huila': 'Andina', 'Norte Santander': 'Andina', 'Quindio': 'Andina',
    'Risaralda': 'Andina', 'Santander': 'Andina', 'Tolima': 'Andina', 'Bogotá': 'Andina',

    # Región Caribe
    'Atlantico': 'Caribe', 'Bolivar': 'Caribe', 'Cesar': 'Caribe', 'Cordoba': 'Caribe',
    'La Guajira': 'Caribe', 'Magdalena': 'Caribe', 'Sucre': 'Caribe','San Andres': 'Caribe',

    # Región Pacífica
    'Cauca': 'Pacífica', 'Choco': 'Pacífica', 'Nariño': 'Pacífica', 'Valle': 'Pacífica',

    # Región Orinoquía
    'Meta': 'Orinoquía', 'Arauca': 'Orinoquía', 'Casanare': 'Orinoquía', 'Vichada': 'Orinoquía',

    # Región Amazónica
    'Caqueta': 'Amazónica', 'Putumayo': 'Amazónica', 'Amazonas': 'Amazónica',
    'Guainia': 'Amazónica', 'Guaviare': 'Amazónica', 'Vaupes': 'Amazónica'
}

# Crear la nueva variable 'region' en el dataframe ICFES
icfes['region'] = icfes['colegio_dpto'].map(departamento_a_region)

# Verificar si hay departamentos sin asignación
print(icfes[icfes['region'].isna()]['colegio_dpto'].unique())

## 7. Eliminar Filas

In [ ]:
var_eli = ["est_mpio_presento_examen", "est_dpto_presento_examen", 
           "fecha_nac", "año_nacimiento", 'pertenece_etnia', "dpto_est_reside", "mpio_est_reside"]

icfes = icfes.drop(columns=var_eli)
icfes.sample(4)

In [ ]:
# Analizar el porcentaje de missing values
missing_analysis = pd.DataFrame({
    'Column': icfes.columns,
    'Missing_Count': icfes.isnull().sum(),
    'Missing_Percentage': (icfes.isnull().sum() / len(icfes)) * 100
}).sort_values('Missing_Percentage', ascending=False)

print(missing_analysis)

In [ ]:
# Análisis por fila
missing_per_row = icfes.isnull().sum(axis=1)
print(f"Filas sin missing: {(missing_per_row == 0).sum()}")
print(f"Filas con 1 missing: {(missing_per_row == 1).sum()}")
print(f"Filas con 2 missing: {(missing_per_row == 2).sum()}")
print(f"Filas con 1-2 missing: {((missing_per_row >= 1) & (missing_per_row <= 2)).sum()}")
print(f"Filas con 3 missing: {(missing_per_row == 3).sum()}")
print(f"Filas con 4 missing: {(missing_per_row == 4).sum()}")
print(f"Filas con 3-4 missing: {((missing_per_row >= 3) & (missing_per_row <= 4)).sum()}")
print(f"Filas con 5 missing: {(missing_per_row == 5).sum()}")
print(f"Filas con >5 missing: {(missing_per_row > 5).sum()}")

In [ ]:
columnas_a_eliminar = ["grupo_edad", "genero"]

filas_antes = icfes.shape[0]

# Eliminar filas con NA en estas columnas
icfes = icfes.dropna(subset=columnas_a_eliminar)

# Mostrar la cantidad de filas restantes
print(f"Cantidad de filas antes de eliminar NA: {filas_antes}")
print("------------------------------------------------------")
print(f"Cantidad de filas después de eliminar NA: {icfes.shape[0]}")
print("------------------------------------------------------")
print(f"Filas eliminadas: {filas_antes - icfes.shape[0]}")

In [ ]:
# Información básica del dataset
total_columnas = icfes.shape[1]
print(f"Total de columnas en el dataset: {total_columnas}")
print("\n📊 CONVERSIÓN DE PORCENTAJES A NÚMEROS ABSOLUTOS")
print("=" * 60)

# Calcular cuántos NAs corresponden a cada porcentaje
porcentajes = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

for pct in porcentajes:
    num_nas = (pct / 100) * total_columnas
    num_nas_entero = int(num_nas)
    num_nas_redondeado = round(num_nas)
    
    print(f"{pct}% de NA = {num_nas:.1f} columnas ≈ {num_nas_redondeado} columnas")

In [ ]:
# Contar NA por fila
na_por_fila = icfes.isna().sum(axis=1)

# Calcular el porcentaje de NA por fila
porcentaje_na_por_fila = (na_por_fila / icfes.shape[1]) * 100

# Información del dataset
total_columnas = icfes.shape[1]

# Calcular equivalencias en números absolutos
nas_10_pct = round((10 / 100) * total_columnas)
nas_20_pct = round((20 / 100) * total_columnas)
nas_25_pct = round((25 / 100) * total_columnas)

# Contar filas con más del 25% de NA
filas_con_mas_25_na = (porcentaje_na_por_fila > 25).sum()

# Estadísticas adicionales
na_promedio_por_fila = porcentaje_na_por_fila.mean()
nas_promedio_absoluto = round((na_promedio_por_fila / 100) * total_columnas)
indice_fila_max_na = na_por_fila.idxmax()
maximo_na = na_por_fila.max()
porcentaje_max_na = porcentaje_na_por_fila.max()

# Resultados con equivalencias
print(f"Dataset tiene {total_columnas} columnas totales")
print("-" * 50)
print(f"Hay {((porcentaje_na_por_fila >= 10) & (porcentaje_na_por_fila < 20)).sum()} filas entre el 10% y 20% de NA (entre {nas_10_pct} y {nas_20_pct-1} columnas faltantes)")
print(f"Hay {(porcentaje_na_por_fila == 20).sum()} filas con exactamente el 20% de NA (exactamente {nas_20_pct} columnas faltantes)")
print(f"Hay {filas_con_mas_25_na} filas con más del 25% de NA (más de {nas_25_pct} columnas faltantes)")
print(f"Promedio de NA por fila: {na_promedio_por_fila:.2f}% (≈ {nas_promedio_absoluto} columnas faltantes)")
print(f"La fila con más NA es la fila {indice_fila_max_na} con {maximo_na} valores NA ({porcentaje_max_na:.2f}%)")

In [ ]:
filas_antes = icfes.shape[0]

# 1. Contar los NA por fila
na_counts_per_row = icfes.isna().sum(axis=1)

# 2. Filtrar las filas con más de 5 NA
rows_to_drop = na_counts_per_row[na_counts_per_row >= 5].index

# 3. Eliminar las filas filtradas
icfes = icfes.drop(rows_to_drop)

print(f"Se eliminaron {len(rows_to_drop)} filas con más de 5 valores NA.")

# Mostrar la cantidad de filas restantes
print(f'Cantidad de filas antes de eliminar NA: {filas_antes}')
print(f"Cantidad de filas después de eliminar NA: {icfes.shape[0]}")
print(f"Filas eliminadas: {filas_antes - icfes.shape[0]}")

In [ ]:
icfes.shape

In [ ]:
icfes.columns

In [ ]:
orden_variables = ['nacionalidad', 'genero', 'region', 'etnia_estudiante', 'edad', 'grupo_edad', 'presento_fuera_edad', 'num_personas_casa', 'num_cuartos_casa', 'estrato_casa',
                   'tiempo_internet', 'internet', 'tv', 'computador', 'lavadora', 'microndas', 'carro', 'moto', 'consola', 'situacion_economica_casa',
                   'num_libros', 'tiempo_lectura', 'freq_leche_derivados', 'freq_carne_pescado_similares', 'freq_cereales_frutos_legumbres',
                   'nivel_edu_padre', 'actividad_padre', 'nivel_edu_madre', 'actividad_madre', 'horas_trabajo_semanal', 'tipo_remuneracion',
                   'colegio_genero', 'colegio_naturaleza', 'colegio_calendario', 'colegio_caracter', 'colegio_area', 'colegio_jornada', 'colegio_mpio', 'colegio_dpto', 'puntaje_lectura',
                   'percentil_lectura', 'puntaje_matematicas', 'percentil_matematicas', 'nivel_matematicas', 'puntaje_naturales', 'percentil_naturales', 'nivel_naturales', 'puntaje_sociales', 
                   'percentil_sociales', 'nivel_sociales', 'puntaje_ingles', 'percentil_ingles', 'nivel_ingles', 'puntaje_global', 'percentil_global', 'inse_estudiante', 
                   'nse_estudiante', 'nse_colegio']

icfes = icfes[orden_variables]

In [ ]:
# Guardar DataFrame en parquet
icfes.to_parquet("icfes_limpio.parquet", index=False, engine="pyarrow")